# Trafficking People Data set
Application of data mining process as a final project for *Data mining* course at _Universidad Nacional de Colombia_.</br>
Profesor on charge _[ELizabeth León Guzmán](https://github.com/elizleon)_ PH. D. at University of Louisville.

Authors of the project
* Juan Manuel Correa Lombana
* Julio Ernesto Quintero Peña

# Classification
